In [25]:
import numpy as np
from finlab import data, backtest

rootpath= "D:/project/finlabexportdata/"
import finlab
finlab.login(open(f"{rootpath}config.txt", "r").read())

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # 忽略FutureWarning

close =  data.get('price:收盤價')
volume = data.get('price:成交股數')
revenue = data.get('monthly_revenue:當月營收')


# 營收成長
c1 = (revenue.pct_change().average(6) > 0).index_str_to_date()

# 通道狹窄
c2 = close.rolling(20).min() > close.rolling(20).max() * 0.7

# 成交量與價格收縮
volumeReduce = volume.average(10) < (volume.average(60)*0.5)
# priceContract2 = close.rolling(30).std() < (close.rolling(60).std()*0.5)
priceContract = close.rolling(10).std() < (close.rolling(60).std()*0.5)

# VCP = 價量收縮、創新高、有足夠成交量
vcp = (volumeReduce & priceContract).sustain(5, 1) & (close == close.rolling(100).max()) & (volume >= volume.rolling(20).mean()*0.8)


# 總體部位
# 進場：(c1, c2, vcp, 流動性足夠, 長期均向上)
# 出場：股價 < 季線
buy = (vcp & c1 & c2 & (volume*close > 2000000) & (close > close.average(250)))
sell = close < close.average(60)

position = buy.hold_until(sell, nstocks_limit=20, rank=(revenue / revenue.average(3)).replace([np.inf, -np.inf], np.nan).index_str_to_date())

# 過濾 KY股
sc = data.get("security_categories")
position_col = position.columns
ky_filter = position_col[~position_col.isin(list(sc[sc["name"].str.contains("KY")]["stock_id"]))]
position = position[ky_filter]


# 回測
r = backtest.sim(position, fee_ratio=1.425/1000/3, resample_offset='D', position_limit=0.2, resample='D', stop_loss=0.1)

輸入成功!
